<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D9%87%D9%8A%D8%AF%D8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def hedge_position_with_retrace_limit(symbol):
    """تطبيق استراتيجية الهيدج مع قيود على عدد الصفقات المفتوحة وحجم الصفقة الجديدة."""
    positions = mt5.positions_get(symbol=symbol)
    open_positions_count = len(positions)

    # تحقق من عدد الصفقات المفتوحة
    if open_positions_count >= 7:
        print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7).")
        return

    if positions:
        for position in positions:
            # تحديد سعر الدخول ووقف الخسارة
            entry_price = position.price_open
            stop_loss = position.sl  # سعر وقف الخسارة

            # حساب المسافة بين سعر الدخول ووقف الخسارة
            distance_to_sl = entry_price - stop_loss if position.type == mt5.ORDER_BUY else stop_loss - entry_price

            # حساب السعر الذي يمثل 75% من المسافة نحو وقف الخسارة
            retrace_price = entry_price - (0.75 * distance_to_sl) if position.type == mt5.ORDER_BUY else entry_price + (0.75 * distance_to_sl)

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).ask

            # إذا كان السعر الحالي قد تراجع بنسبة 75% نحو وقف الخسارة
            if (position.type == mt5.ORDER_BUY and current_price <= retrace_price) or \
               (position.type == mt5.ORDER_SELL and current_price >= retrace_price):

                # فتح مركز معكوس إذا كان العدد أقل من 7
                hedge_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,  # نفس حجم المركز الحالي
                    "type": mt5.ORDER_SELL if position.type == mt5.ORDER_BUY else mt5.ORDER_BUY,  # نوع الصفقة المعكوسة
                    "price": mt5.symbol_info_tick(symbol).bid if position.type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).ask,  # السعر الحالي
                    "sl": 0,  # يمكنك تعيين وقف الخسارة إذا لزم الأمر
                    "tp": 0,  # يمكنك تعيين جني الأرباح إذا لزم الأمر
                    "magic": 123456,  # رقم سحري فريد
                    "comment": "Hedging position on retrace",
                }
                result = mt5.order_send(hedge_request)
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"فشل في فتح صفقة الهيدج: {result.retcode}")

    # إذا كان عدد الصفقات أقل من 7 و5 أو أقل، افتح صفقة جديدة بنفس حجم الصفقة القديمة
    if open_positions_count < 7 and open_positions_count <= 5:
        # استخدام حجم الصفقة القديمة
        last_position_volume = positions[-1].volume if positions else 1.0  # إذا لم توجد صفقات، استخدم حجم افتراضي

        # فتح صفقة جديدة هنا
        new_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": last_position_volume,  # حجم الصفقة الجديدة هو نفس حجم الصفقة القديمة
            "type": mt5.ORDER_BUY,  # نوع الصفقة الجديدة (يمكن تغييره حسب الحاجة)
            "price": mt5.symbol_info_tick(symbol).ask,  # السعر الحالي
            "sl": 0,  # يمكنك تعيين وقف الخسارة إذا لزم الأمر
            "tp": 0,  # يمكنك تعيين جني الأرباح إذا لزم الأمر
            "magic": 123456,  # رقم سحري فريد
            "comment": "New position to reach 7 trades",
        }
        new_result = mt5.order_send(new_request)
        if new_result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"فشل في فتح صفقة جديدة: {new_result.retcode}")
